[db_schema]<img src="files/db.png">

1.
Calculate how many companies have closed down.

In [ ]:
SELECT COUNT(name)
FROM company
WHERE status='closed'
GROUP BY status;

2.
Print the amount of money raised for news companies in the US. Use data from the company table. Sort the resulting table in descending order of value in the funding_total field.

In [ ]:
select funding_total 
from company
WHERE country_code = 'USA'
AND category_code LIKE '%news%'
ORDER BY funding_total DESC;

3.
Find the total price of the purchases of companies by other organizations in USD. Select the transactions that were carried out only in cash from 2011 to 2013 inclusive.

In [ ]:
select sum(price_amount)
from acquisition
WHERE term_code = 'cash'
AND EXTRACT(YEAR FROM CAST(acquired_at as date)) BETWEEN 2011 and 2013

4.
Print names, last names, and the Twitter accounts of the people whose Twitter usernames begin with 'Silver'.

In [ ]:
select first_name,last_name,twitter_username
from people
where twitter_username LIKE 'Silver%'

5.
Print all information about the people whose Twitter usernames contain the substring 'money' and whose last name begins with 'K'.

In [ ]:
select * 
from people
where twitter_username LIKE '%money%'
and last_name LIKE 'K%'

6.
For each country, print the total investment that companies registered in that country have received. Sort the data in descending order.

In [ ]:
select country_code,
       sum(funding_total) as rounds
from company
group by country_code
order by rounds DESC;

7.
Create a table that includes the date the record was created in the table, together with the minimum and maximum values of the attracted investments.
In addition to that, leave all records in the table where the minimum value of the investment is not zero and is also not equal to the maximum funding value.

In [ ]:
select CAST(created_at AS date),
       MIN(funding_total) as min_funding,
       MAX(funding_total) as max_funding
from company
group by CAST(created_at AS date)
HAVING MIN(funding_total)!=0
and MIN(funding_total)!= MAX(funding_total);

8.
Create a field with these categories:
high_activity — funds that invest in 100 or more companies
middle_activity — funds that invest in 20 to 100 companies
low_activity — funds that invest in fewer than 20 companies
Print all the fields in the fund table and the new category field.

In [ ]:
select *,
        CASE
            WHEN invested_companies > 100 THEN 'high_activity' 
            WHEN invested_companies BETWEEN 20 and 100 THEN 'middle_activity' 
            WHEN invested_companies <20 THEN 'low_activity'
         END as activity
from fund;

9.
For each of the categories assigned in the previous task, calculate the average number of investment rounds in which the fund participated, rounded to the nearest integer. Print the categories and the average number of investment rounds. Sort the table in ascending order of the average.

In [ ]:
SELECT activity,
       ROUND(AVG(investment_rounds)) as avg_round
FROM(
    select *,
            CASE
                WHEN invested_companies > 100 THEN 'high_activity' 
                WHEN invested_companies BETWEEN 20 and 100 THEN 'middle_activity' 
                WHEN invested_companies <20 THEN 'low_activity'
             END as activity
    from fund
    ) as f
GROUP BY activity
ORDER BY avg_round;

10.
Get a table with the ten most actively investing countries. Determine the activity of a country by the average number of companies in which the funds of this country invest.
For each country, calculate the minimum, maximum, and average number of companies in which funds founded from 2010 to 2012, inclusive, have invested. Make sure to take only the year part of the date.
Exclude from the table the countries with funds that have the minimum number of invested companies equal to zero.

In [ ]:
 select  country_code,
         MIN(invested_companies),
         MAX(invested_companies),
         AVG(invested_companies) as avg
         
 from fund
 WHERE EXTRACT(YEAR FROM CAST(founded_at as date)) BETWEEN 2010 and 2012
 group by country_code
 having MIN(invested_companies) >0
 order by avg desc
 limit 10;

11.
Print names and last names of all startup employees. Add a field with the name of the educational institution that the employee graduated from if this information is known.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people as p
LEFT JOIN education as e ON p.id = e.person_id

12.
For each company, find the number of educational institutions that its employees graduated from. Print the names of the companies and the number of unique names of educational institutions. Create a list of the top 5 companies by the number of universities.

In [ ]:
WITH
c AS (
      SELECT id,
              name
       FROM company),
p AS (
       SELECT company_id,
               id
        FROM people),
e AS (SELECT person_id,
             instituition
     FROM education)
 SELECT name,
        COUNT(DISTINCT instituition) 
 FROM c
 join p ON c.id = p.company_id
 join e on p.id = e.person_id
 group by name
 order by count DESC
 limit 5;

13.
Create a list of the closed companies for which the first round of funding was the last one.

In [ ]:
WITH 
c AS (  SELECt id,name,status
        FROM company
        WHERE status LIKE '%closed%'
        ),
f AS (SELECT company_id,
             is_first_round,
             is_last_round
      FROM funding_round
      WHERE is_first_round = 1
        AND is_last_round = 1
      )
 SELECT DISTINCT name
 FROM c
 inner join f ON c.id = f.company_id

14.
Create a list of IDs of the employees that work for the companies selected in the previous task.

In [ ]:
WITH 
c1 AS (WITH 
        c AS (  SELECt id,name,status
                FROM company
                WHERE status LIKE '%closed%'
                ),
        f AS (SELECT company_id,
                     is_first_round,
                     is_last_round
              FROM funding_round
              WHERE is_first_round = 1
                AND is_last_round = 1
              )
         SELECT DISTINCT name
         FROM c
         inner join f ON c.id = f.company_id
      ),
c2 AS (SELECT name,id
       FROM company
      ),
p AS (SELECT id as p_id
      ,company_id
     FROM people)      
       
SELECT p_id
FROM c1
LEFT JOIN c2 ON c1.name = c2.name
INNER JOIN p on c2.id = p.company_id

15.
Create a table with the IDs of the employees from the previous task together with the educational institution they have graduated from.

In [ ]:
SELECT person_id,instituition
FROM (
        WITH 
        c1 AS (WITH 
                c AS (SELECt id,name,status
                      FROM company
                      WHERE status LIKE '%closed%'
                      ),
                f AS (SELECT company_id,
                             is_first_round,
                             is_last_round
                      FROM funding_round
                      WHERE is_first_round = 1
                        AND is_last_round = 1
                      )
                 SELECT DISTINCT name
                 FROM c
                 inner join f ON c.id = f.company_id
              ),
        c2 AS (SELECT name,id
               FROM company
              ),
        p AS (SELECT id as p_id
              ,company_id
             FROM people)      

        SELECT p_id
        FROM c1
        LEFT JOIN c2 ON c1.name = c2.name
        INNER JOIN p on c2.id = p.company_id
    ) as e_ids
    INNER JOIN education as e ON e_ids.p_id = e.person_id

16.
Count the number of educational institutions for each employee from the previous task.

In [ ]:
SELECT person_id,
       COUNT(instituition)
FROM(       
        SELECT person_id,instituition
        FROM (
                WITH 
                c1 AS (WITH 
                        c AS (SELECt id,name,status
                              FROM company
                              WHERE status LIKE '%closed%'
                              ),
                        f AS (SELECT company_id,
                                     is_first_round,
                                     is_last_round
                              FROM funding_round
                              WHERE is_first_round = 1
                                AND is_last_round = 1
                              )
                         SELECT DISTINCT name
                         FROM c
                         inner join f ON c.id = f.company_id
                      ),
                c2 AS (SELECT name,id
                       FROM company
                      ),
                p AS (SELECT id as p_id
                      ,company_id
                     FROM people)      

                SELECT p_id
                FROM c1
                LEFT JOIN c2 ON c1.name = c2.name
                INNER JOIN p on c2.id = p.company_id
            ) as e_ids
            INNER JOIN education as e ON e_ids.p_id = e.person_id
) as p
GROUP BY person_id;

17.
Print the average number of educational institutions that employees of different companies have graduated from. Just print one record, there is no need for grouping here

Use a subquery to find the average of the data you got from the statement in the previous task

In [ ]:
SELECT AVG(count)
FROM (
        SELECT person_id,
               COUNT(instituition)
        FROM(       
                SELECT person_id,instituition
                FROM (
                        WITH 
                        c1 AS (WITH 
                                c AS (SELECt id,name,status
                                      FROM company
                                      WHERE status LIKE '%closed%'
                                      ),
                                f AS (SELECT company_id,
                                             is_first_round,
                                             is_last_round
                                      FROM funding_round
                                      WHERE is_first_round = 1
                                        AND is_last_round = 1
                                      )
                                 SELECT DISTINCT name
                                 FROM c
                                 inner join f ON c.id = f.company_id
                              ),
                        c2 AS (SELECT name,id
                               FROM company
                              ),
                        p AS (SELECT id as p_id
                              ,company_id
                             FROM people)      

                        SELECT p_id
                        FROM c1
                        LEFT JOIN c2 ON c1.name = c2.name
                        INNER JOIN p on c2.id = p.company_id
                    ) as e_ids
                    INNER JOIN education as e ON e_ids.p_id = e.person_id
        ) as p
        GROUP BY person_id
    ) as p2;

18.
Print the average number of educational institutions that Facebook employees graduated from.

In [ ]:
SELECT AVG(count)
FROM (
        WITH
         p AS (SELECT id 
                FROM people
                WHERE company_id = 5
             ),
         e AS (SELECT person_id,
                      instituition
              FROM education)
        SELECT person_id,
               COUNT(instituition)
        FROM p 
        INNER JOIN e ON p.id = e.person_id
        GROUP BY person_id
    ) as fb


19.
Create a table that consists of the following fields:
name_of_fund — the name of the fund
name_of_company — the name of the company
amount — the investments that the company attracted in the round
The table must include data on companies with more than six important milestones in their history, and that had funding rounds that took place in 2012 and 2013.

In [ ]:
select  f.name as name_of_fund ,
        c.name as name_of_company ,
       --i.company_id,
       --i.fund_id,       
         fr.raised_amount
from investment as i
right join company as c ON i.company_id = c.id
left join fund as f ON i.fund_id = f.id
left join funding_round as fr ON i.funding_round_id =fr.id
where c.milestones >6
and EXTRACT(YEAR FROM CAST(fr.funded_at as date)) IN (2012,2013)

20.
Get a table with the following fields:
The name of the buying company
The price of the purchase
The name of the company that was bought
The investments invested in the purchased company
The date when the company was acquired
A ratio that shows how many times the purchase price exceeded the investments in the company, rounded up to the nearest integer.
Do not count those purchases in which the purchase price is equal to zero. Apart from that, if the investment in an acquired company is zero, exclude that company from the table too.
Sort the table by the purchase price from highest to lowest. Limit the table to the first ten entries.

In [ ]:
select c.name as acquiring_company,
       --acquiring_company_id,
       --acquired_company_id,
       c2.name as acquired_company,
       a.price_amount,
       c2.funding_total,
       a.acquired_at,
       ROUND(a.price_amount/c2.funding_total)
from acquisition as a
inner join company as c ON a.acquiring_company_id = c.id
inner join company as c2 ON a.acquired_company_id = c2.id
where a.price_amount >0
and c2.funding_total >0
order by a.price_amount desc
limit 10;

21.
Get a table with the names of companies from the social category that received funding from 2010 to 2013, together with the number of the month when the funding round took place.

In [ ]:
select c.name,
       --c.category_code,
       EXTRACT(MONTH FROM CAST(fr.funded_at as date)) as month
from company as c
inner join funding_round as fr ON c.id = fr.company_id
where category_code LIKE '%social%'
 and EXTRACT(YEAR FROM CAST(fr.funded_at as date)) BETWEEN 2010 and 2013

22.
For each month from 2010 to 2013, find the number of unique funds registered in the US.
Get a table that contains the following fields:
The number of the month
The number of unique funds from the US
The number of purchases
The total turnover of transactions per month

In [ ]:
WITH
a as (select EXTRACT(MONTH FROM CAST(fr.funded_at as date)) as month,
       COUNT(DISTINCT f.id) as unique_us_funds     
       --sum(fr.raised_amount) as total_of_purchases
        from fund as f
        join investment as i on f.id = i.fund_id
        join funding_round as fr on i.funding_round_id = fr.id
        where f.country_code = 'USA'
        and EXTRACT(YEAR FROM CAST(fr.funded_at as date)) BETWEEN 2010 and 2013
        group by month),
b as (select EXTRACT(MONTH FROM CAST(acquired_at as date)) as month,
               COUNT(acquired_company_id),
               SUM(price_amount)
        from acquisition
        WHERE EXTRACT(YEAR FROM CAST(acquired_at as date)) BETWEEN 2010 and 2013
        GROUP BY month)
 SELECT a.month,
        a.unique_us_funds,
        b.count,
        b.sum
 FROM a join b on a.month = b.month

23.
Create a table with the average investment for each country in 2011, 2012, and 2013. Print the data for each year in a separate field. Sort the table by 2011 average investment from highest to lowest.
Note that we only want to consider countries with data across all three of these years.

In [ ]:
-- the one
WITH
el as (select country_code,
          avg(funding_total)
       from company
       where EXTRACT(YEAR FROM CAST(founded_at as date)) = 2011
       group by country_code
       ),
tw as (select country_code,
               avg(funding_total)
        from company
        where EXTRACT(YEAR FROM CAST(founded_at as date)) = 2012
        group by country_code),
th as (select country_code,
              avg(funding_total)
       from company
       where EXTRACT(YEAR FROM CAST(founded_at as date)) = 2013
       group by country_code)
SELECT el.country_code,
       el.avg as year_2011,
       tw.avg as year_2012,
       th.avg as year_2013
from el
join tw on el.country_code = tw.country_code
join th on tw.country_code = th.country_code
order by year_2011	desc;